# Group Assignment PBD PL1 - Multinomial Logistic Regression & Decision Tree Classifier
## Book 2 of 3: Multinomial Logistic Regression


Authors:

Catarina Castanheira Nº 92478

João Martins Nº 93259

Joel Paula Nº 93392



Read small dataset from parquet filet.

In [1]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # pretty print tables
df = spark.read.format("parquet").load("collisions_clean")

In [2]:
df.printSchema()

root
 |-- jurisdiction: string (nullable = true)
 |-- chp_shift: string (nullable = true)
 |-- population: string (nullable = true)
 |-- county_city_location: string (nullable = true)
 |-- special_condition: string (nullable = true)
 |-- beat_type: string (nullable = true)
 |-- chp_beat_type: string (nullable = true)
 |-- city_division_lapd: string (nullable = true)
 |-- chp_beat_class: string (nullable = true)
 |-- beat_number: string (nullable = true)
 |-- primary_road: string (nullable = true)
 |-- secondary_road: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- direction: string (nullable = true)
 |-- intersection: long (nullable = true)
 |-- weather_1: string (nullable = true)
 |-- weather_2: string (nullable = true)
 |-- state_highway_indicator: long (nullable = true)
 |-- caltrans_county: string (nullable = true)
 |-- caltrans_district: string (nullable = true)
 |-- state_route: string (nullable = true)
 |-- route_suffix: string (nullable = true)
 |-- postmi

In [3]:
df.limit(30)

jurisdiction,chp_shift,population,county_city_location,special_condition,beat_type,chp_beat_type,city_division_lapd,chp_beat_class,beat_number,primary_road,secondary_road,distance,direction,intersection,weather_1,weather_2,state_highway_indicator,caltrans_county,caltrans_district,state_route,route_suffix,postmile_prefix,postmile,location_type,ramp_intersection,side_of_highway,tow_away,collision_severity,killed_victims,injured_victims,party_count,primary_collision_factor,pcf_violation_code,pcf_violation_category,pcf_violation,pcf_violation_subsection,hit_and_run,type_of_collision,motor_vehicle_involved_with,pedestrian_action,road_surface,road_condition_1,road_condition_2,lighting,control_device,chp_road_type,pedestrian_collision,bicycle_collision,motorcycle_collision,truck_collision,alcohol_involved,statewide_vehicle_type_at_fault,chp_vehicle_type_at_fault,severe_injury_count,other_visible_injury_count,complaint_of_pain_injury_count,pedestrian_killed_count,pedestrian_injured_count,bicyclist_killed_count,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,collision_date,collision_time
9495,1,9,1600,0,1,interstate,Not Stated,chp other,004,RT 5,RT 41,2640.0,south,0,cloudy,Not Stated,1,KIN,6,5,Not Stated,Not Stated,16.07,highway,Not Stated,northbound,1,property damage only,0,0,2,vehicle code viol...,Not Stated,improper turning,22107,Not Stated,not hit and run,sideswipe,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,none,1,0,0,0,1,0,passenger car,07,0,0,0,0,0,0,0,0,0,2008-01-07,11:04:00
9265,1,9,3900,1,3,county road area,Not Stated,chp other,010,KAISER RD,MARIPOSA RD,1584.0,north,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,property damage only,0,0,1,other than driver,Not Stated,other than driver...,Not Stated,Not Stated,not hit and run,hit object,other object,no pedestrian inv...,wet,other,Not Stated,dark with no stre...,none,0,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-07,06:50:00
4807,5,6,4807,0,0,not chp,Not Stated,not chp,006,MISSISSIPPI ST,SACRAMENTO ST,114.0,east,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,property damage only,0,0,2,vehicle code viol...,Not Stated,dui,23152,A,not hit and run,sideswipe,parked motor vehicle,no pedestrian inv...,dry,normal,Not Stated,dark with street ...,none,0,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-15,23:30:00
9725,2,9,4300,0,1,state route,Not Stated,chp primary,160,RT 152,FERGUSON RD,280.0,west,0,Not Stated,Not Stated,1,SCL,4,152,Not Stated,Not Stated,12.76,highway,Not Stated,westbound,0,property damage only,0,0,2,vehicle code viol...,Not Stated,improper turning,22107,Not Stated,not hit and run,broadside,other motor vehicle,no pedestrian inv...,Not Stated,Not Stated,Not Stated,Not Stated,functioning,0,0,0,0,1,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-04,14:45:00
3000,5,4,3021,0,0,not chp,Not Stated,not chp,023,BEACH BL,CHAPMAN,0.0,Not Stated,1,clear,Not Stated,1,ORA,12,39,Not Stated,Not Stated,9.671,intersection,5,northbound,1,pain,0,2,2,vehicle code viol...,Not Stated,traffic signals a...,21453,A,not hit and run,Not Stated,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,functioning,0,0,0,0,0,0,passenger car,01,0,0,2,0,0,0,0,0,0,2008-01-01,12:40:00
9450,1,9,2000,0,1,state route,Not Stated,chp other,012,RT 99,AVENUE 24 1/2,550.0,north,0,cloudy,Not Stated,1,MAD,6,99,Not Stated,Not Stated,24.53,highway,Not Stated,northbound,1,property damage only,0,0,1,other than driver,Not Stated,other than driver...,Not Stated,Not Stated,not hit and run,hit object,other object,no pedestrian inv...,wet,normal,Not Stated,daylight,none,1,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-04,09:21:00
9580,1,7,1942,0,1,us highway,Not Stated,chp other,102,RT 101,HASKELL AV,215.0,north,0,clear,Not Stated,1,LA,7,101,Not Stated,Not Stated,17.535,r

Imports we will need for our work.

In [3]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline

Functions we will use for every model.

In [4]:
def showModelSummary(trainingSummary):
    accuracy = trainingSummary.accuracy
    falsePositiveRate = trainingSummary.weightedFalsePositiveRate
    truePositiveRate = trainingSummary.weightedTruePositiveRate
    fMeasure = trainingSummary.weightedFMeasure()
    precision = trainingSummary.weightedPrecision
    recall = trainingSummary.weightedRecall
    print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
          % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

def showPerformanceOnTest(df_pred, default_prop):
    evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol=target_var_index)
    acc = evaluator.evaluate(df_pred, {evaluator.metricName: "accuracy", })
    print("Accuracy on test set: ", acc)
    # Huberty Index
    hix = (acc - default_prop)/(1 - default_prop)
    print("Huberty Index:", hix)
    return acc
    # (subsetAccuracy|accuracy|hammingLoss|precision|recall|f1Measure|precisionByLabel|recallByLabel|f1MeasureByLabel|microPrecision|microRecall|microF1Measure)

Vectorize all features. But for categorical features, we start by indexing them.

In [5]:
target_var = "collision_severity"
target_var_index = target_var+"_index"
target_var_ohe = target_var+"_ohe"
cols = df.columns
if target_var in cols:
    cols.remove(target_var)

col_names_categorical = [col for (col, dataType) in df.dtypes if dataType == 'string' and col != target_var]
indexOutputCols = [col+"_index" for col in col_names_categorical]
oheOutputCols = [x + "OHE" for x in col_names_categorical]
stringIndexer = StringIndexer(inputCols=col_names_categorical+[target_var], outputCols=indexOutputCols+[target_var_index], handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols, outputCols=oheOutputCols)

col_names_numerical = [col for col in cols if col not in col_names_categorical] 
assembler = VectorAssembler(inputCols=oheOutputCols+col_names_numerical, outputCol="features")


Split train and test sets.

In [6]:
df_test, df_train = df.randomSplit([0.25,0.75], seed=42)
[df_train.count(), df_test.count()]

[75790, 25418]

In [8]:
df_train.cache()
df_test.cache()

jurisdiction,chp_shift,population,county_city_location,special_condition,beat_type,chp_beat_type,city_division_lapd,chp_beat_class,beat_number,primary_road,secondary_road,distance,direction,intersection,weather_1,weather_2,state_highway_indicator,caltrans_county,caltrans_district,state_route,route_suffix,postmile_prefix,postmile,location_type,ramp_intersection,side_of_highway,tow_away,collision_severity,killed_victims,injured_victims,party_count,primary_collision_factor,pcf_violation_code,pcf_violation_category,pcf_violation,pcf_violation_subsection,hit_and_run,type_of_collision,motor_vehicle_involved_with,pedestrian_action,road_surface,road_condition_1,road_condition_2,lighting,control_device,chp_road_type,pedestrian_collision,bicycle_collision,motorcycle_collision,truck_collision,alcohol_involved,statewide_vehicle_type_at_fault,chp_vehicle_type_at_fault,severe_injury_count,other_visible_injury_count,complaint_of_pain_injury_count,pedestrian_killed_count,pedestrian_injured_count,bicyclist_killed_count,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,collision_date,collision_time
100,5,4,0198,0,0,not chp,Not Stated,not chp,2074,DUBLIN BL,CAMP PARKS BL,0.0,Not Stated,1,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,pain,0,1,2,vehicle code viol...,Not Stated,automobile right ...,21453,C,not hit and run,broadside,bicycle,no pedestrian inv...,dry,normal,Not Stated,daylight,functioning,0,0,1,0,0,0,Not Stated,Not Stated,0,0,1,0,0,0,1,0,0,2004-06-08,16:25:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,DPS,BLOOMINGTON WY,BLOOMINGTON WY 11478,0.0,Not Stated,2,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,severe injury,0,1,2,vehicle code viol...,Not Stated,improper turning,22103,Not Stated,not hit and run,broadside,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,none,0,0,0,1,0,0,passenger car,Not Stated,1,0,0,0,0,0,0,0,1,2003-05-11,11:30:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,DUBLIN,DUBLIN BL,AMADOR PLAZA RD,400.0,east,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,property damage only,0,0,2,unknown,Not Stated,unknown,Not Stated,Not Stated,misdemeanor,rear end,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,none,0,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2007-10-18,14:10:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,NOVALU,DUBLIN BL,INSPIRATION DR,5.0,west,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,1,property damage only,0,0,2,vehicle code viol...,Not Stated,speeding,22350,Not Stated,not hit and run,rear end,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,functioning,0,0,0,0,0,0,passenger car,01,0,0,0,0,0,0,0,0,0,2019-03-17,07:44:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,Not Stated,AMADOR PLAZA BL,DUBLIN BL,245.0,north,0,cloudy,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,1,property damage only,0,0,2,vehicle code viol...,Not Stated,automobile right ...,21804,A,not hit and run,broadside,other motor vehicle,no pedestrian inv...,dry,Not Stated,Not Stated,daylight,none,0,0,0,0,0,0,passenger car wit...,01,0,0,0,0,0,0,0,0,0,2012-03-25,13:33:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,Not Stated,BETLEN DR,PROW WY,32.0,south,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,1,property damage only,0,0,1,vehicle code viol...,Not Stated,improper turning,22107,Not Stated,not hit and run,hit object,fixed object,no pedestrian inv...,dry,normal,Not Stated,daylight,none,0,0,0,0,0,0,passenger car,01,0,0,0,0,0,0,0,0,0,2016-08-02,10:58:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,Not Stated,DOUGHERTY R

Take a look at our indexed target classes.

In [7]:
dist_coll_sev = df_test.groupBy("collision_severity").count().toPandas()
dist_coll_sev

,collision_severity,count
0,property damage only,15293
1,pain,6205
2,other injury,3171
3,fatal,192
4,severe injury,557


Store default proportion (majority class frequency) for model evaluation later on.

In [8]:
default_p = dist_coll_sev.sort_values(["count"], ascending=False).iloc[0,1] / sum(dist_coll_sev["count"])

Create the Logistic regression model from the train test set. Create the pipeline.

In [11]:
lr = LogisticRegression(featuresCol='features', labelCol=target_var_index, family="multinomial", maxIter=10, regParam=0.3, elasticNetParam=0.8)

pipeline = Pipeline(stages=[stringIndexer, oheEncoder, assembler, lr])

# Fit the pipeline to training documents.
lrModel = pipeline.fit(df_train)

Have a look at the performance metrics of the model. Then, apply it to the test set and evaluate again.

In [12]:
showModelSummary(lrModel.stages[3].summary)
df_preds = lrModel.transform(df_test)
accuracy = showPerformanceOnTest(df_preds, default_p)

Accuracy: 0.6273782820952633
FPR: 0.5562992517625379
TPR: 0.6273782820952631
F-measure: 0.5054799152270376
Precision: 0.6039998876492404
Recall: 0.6273782820952631
Accuracy on test set:  0.6212919138724085
Huberty Index: 0.04928373993174094


Let's try a model with only the most promising features (those who have a correlation higher than 0.20 with our target).

In [9]:
most_promising_features = ["injured_victims", "other_visible_injury_count", 
                           "severe_injury_count", "killed_victims", "pedestrian_collision", 
                           "tow_away", "motorcyclist_injured_count"]
stringIndexer = StringIndexer(inputCols=[target_var], outputCols=[target_var_index], handleInvalid="skip")
assembler = VectorAssembler(inputCols=most_promising_features, outputCol="mp2_features")

lr = LogisticRegression(featuresCol='mp2_features', labelCol=target_var_index, family="multinomial", maxIter=10, regParam=0.3, elasticNetParam=0.8)

pipeline = Pipeline(stages=[stringIndexer, assembler, lr])

# Fit the pipeline to training documents.
lrModel2 = pipeline.fit(df_train)

showModelSummary(lrModel2.stages[2].summary)
df_pred = lrModel2.transform(df_test)
accuracy = showPerformanceOnTest(df_pred, default_p)

Accuracy: 0.6273123103311783
FPR: 0.5568299913634116
TPR: 0.6273123103311782
F-measure: 0.5052913597018264
Precision: 0.6192845099032884
Recall: 0.6273123103311782
Accuracy on test set:  0.6223935793532143
Huberty Index: 0.0520493827160493


We achieved no Accuracy improvement with less variables. 

Let's try only with variables that have a high correlation with the target variable.

In [10]:
most_promising_features = ["injured_victims", "other_visible_injury_count", "severe_injury_count"]
stringIndexer = StringIndexer(inputCols=[target_var], outputCols=[target_var_index], handleInvalid="skip")
assembler = VectorAssembler(inputCols=most_promising_features, outputCol="mp3_features")

lr = LogisticRegression(featuresCol='mp3_features', labelCol=target_var_index, family="multinomial", maxIter=10, regParam=0.3, elasticNetParam=0.8)


pipeline = Pipeline(stages=[stringIndexer, assembler, lr])

# Fit the pipeline to training documents.
lrModel3 = pipeline.fit(df_train)

showModelSummary(lrModel3.stages[2].summary)
df_pred = lrModel3.transform(df_test)
accuracy = showPerformanceOnTest(df_pred, default_p)

Accuracy: 0.6273123103311783
FPR: 0.5568299913634116
TPR: 0.6273123103311782
F-measure: 0.5052913597018264
Precision: 0.6192845099032884
Recall: 0.6273123103311782
Accuracy on test set:  0.6223935793532143
Huberty Index: 0.0520493827160493


No improvement achieved.

Our prediction model has around 4% improvement over just picking the most common class. Which means it's not a great model.